# PyCity Schools Analysis

*There is a clear divide between this data set’s charter and district passing rates: every charter school’s average passing rate is greater than 92% and every district school’s average passing rate is less than 82%.

*Although spending per student has some correlation to passing rates in this data set, size appears to have a stronger correlation to passing rates in this: every charter school has fewer students than every district school.  

*The average passing rate for reading has a smaller range (97.309 – 79.299 = 18.010) than the average passing rate for math (94.595 – 65.684 = 28.911).

---

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# Calculate required information.
num_schools = len(school_data_complete["school_name"].unique())
num_students = len(school_data_complete["Student ID"])
total_budget = school_data["budget"].sum()
math_percent = school_data_complete["math_score"].mean()
reading_percent = school_data_complete["reading_score"].mean()
overall_percent = (math_percent + reading_percent) / 2
num_pass_math = len(school_data_complete.loc[school_data_complete["math_score"] >= 70])
num_pass_reading = len(school_data_complete.loc[school_data_complete["reading_score"] >= 70])


In [3]:
# Store information in a data frame and format each column.
District_Sum_df = pd.DataFrame({
    "Total Schools": [num_schools],
    "Total Students": [num_students],
    "Total Budget": ["${:,.2f}".format(total_budget)],
    "Average Math Score": ["{:.3f}".format(math_percent)],
    "Average Reading Score": ["{:.3f}".format(reading_percent)],
    "% Passing Math": ["{:.3f}".format(num_pass_math / num_students * 100)],
    "% Passing Reading": ["{:.3f}".format(num_pass_reading / num_students * 100)],
    "% Overall Passing Rate": ["{:.3f}".format(overall_percent)]
    })

District_Sum_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$24,649,428.00",78.985,81.878,74.981,85.805,80.432


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [4]:
students_by_school = student_data.groupby(["school_name"])

In [5]:
# Start by using the original school data and rename the columns.
School_Sum_df = school_data
School_Sum_df = School_Sum_df.rename(columns={"school_name":"School Name", 
                                              "type": "School Type", 
                                              "size": "Total Students",
                                              "budget": "Total School Budget"})
School_Sum_df = School_Sum_df.drop(columns=["School ID"])
School_Sum_df = School_Sum_df.set_index("School Name")

# Calculate the budget per student.
budget_per_student_df = School_Sum_df["Total School Budget"] / School_Sum_df["Total Students"]

# Calculate required information and store in the data frame.
School_Sum_df["Per Student Budget"] = budget_per_student_df
School_Sum_df["Average Math Score"] = students_by_school["math_score"].mean()
School_Sum_df["Average Reading Score"] = students_by_school["reading_score"].mean()

# Pandas cannot use values from a groupby object directly for calculations.  
# The .apply() function is used to manipulate the data in a groupby object.
School_Sum_df["% Passing Math"] = students_by_school["math_score"].apply(lambda x: (x >= 70).sum()) / School_Sum_df["Total Students"] * 100
School_Sum_df["% Passing Reading"] = students_by_school["reading_score"].apply(lambda x: (x >= 70).sum()) / School_Sum_df["Total Students"] * 100
School_Sum_df["Overall Passing Rate"] = (School_Sum_df["% Passing Math"] + School_Sum_df["% Passing Reading"]) / 2

# Format the data using .apply().
School_Sum_df["Per Student Budget"] = School_Sum_df["Per Student Budget"].apply(lambda x: "${:,.2f}".format(x))
School_Sum_df["Average Math Score"] = School_Sum_df["Average Math Score"].apply(lambda x: "{:.3f}".format(x))
School_Sum_df["Average Reading Score"] = School_Sum_df["Average Reading Score"].apply(lambda x: "{:.3f}".format(x))
School_Sum_df["% Passing Math"] = School_Sum_df["% Passing Math"].apply(lambda x: "{:.3f}".format(x))
School_Sum_df["% Passing Reading"] = School_Sum_df["% Passing Reading"].apply(lambda x: "{:.3f}".format(x))
School_Sum_df["Overall Passing Rate"] = School_Sum_df["Overall Passing Rate"].apply(lambda x: "{:.3f}".format(x))

# Print the data frame.
School_Sum_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Huang High School,District,2917,1910635,$655.00,76.629,81.183,65.684,81.316,73.500
Figueroa High School,District,2949,1884411,$639.00,76.712,81.158,65.988,80.739,73.364
Shelton High School,Charter,1761,1056600,$600.00,83.359,83.726,93.867,95.855,94.861
Hernandez High School,District,4635,3022020,$652.00,77.290,80.934,66.753,80.863,73.808
Griffin High School,Charter,1468,917500,$625.00,83.351,83.817,93.392,97.139,95.266
Wilson High School,Charter,2283,1319574,$578.00,83.274,83.989,93.868,96.540,95.204
Cabrera High School,Charter,1858,1081356,$582.00,83.062,83.976,94.133,97.040,95.587
Bailey High School,District,4976,3124928,$628.00,77.048,81.034,66.680,81.933,74.307
Holden High School,Charter,427,248087,$581.00,83.803,83.815,92.506,96.253,94.379


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [6]:
School_Sum_df.sort_values("Overall Passing Rate", ascending=False).head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,$582.00,83.062,83.976,94.133,97.040,95.587
Thomas High School,Charter,1635,1043130,$638.00,83.418,83.849,93.272,97.309,95.291
Pena High School,Charter,962,585858,$609.00,83.840,84.045,94.595,95.946,95.270
Griffin High School,Charter,1468,917500,$625.00,83.351,83.817,93.392,97.139,95.266
Wilson High School,Charter,2283,1319574,$578.00,83.274,83.989,93.868,96.540,95.204


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [7]:
School_Sum_df.sort_values("Overall Passing Rate").head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,3999,2547363,$637.00,76.843,80.745,66.367,80.220,73.293
Figueroa High School,District,2949,1884411,$639.00,76.712,81.158,65.988,80.739,73.364
Huang High School,District,2917,1910635,$655.00,76.629,81.183,65.684,81.316,73.500
Johnson High School,District,4761,3094650,$650.00,77.072,80.966,66.058,81.222,73.640
Ford High School,District,2739,1763916,$644.00,77.103,80.746,68.310,79.299,73.804


## Math Scores by Grade

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [8]:
# Group students by grade level, and then find the mean values for each school.
grade9 = student_data.loc[student_data["grade"] == "9th"].groupby(["school_name"])
grade9_math_scores = grade9["math_score"].mean()

grade10 = student_data.loc[student_data["grade"] == "10th"].groupby(["school_name"])
grade10_math_scores = grade10["math_score"].mean()

grade11 = student_data.loc[student_data["grade"] == "11th"].groupby(["school_name"])
grade11_math_scores = grade11["math_score"].mean()

grade12 = student_data.loc[student_data["grade"] == "12th"].groupby(["school_name"])
grade12_math_scores = grade12["math_score"].mean()

math_df = pd.DataFrame({
    "9th": grade9_math_scores.apply(lambda x: "{:.3f}".format(x)),
    "10th": grade10_math_scores.apply(lambda x: "{:.3f}".format(x)),
    "11th": grade11_math_scores.apply(lambda x: "{:.3f}".format(x)),
    "12th": grade12_math_scores.apply(lambda x: "{:.3f}".format(x))
})
math_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.084,76.997,77.516,76.492
Cabrera High School,83.095,83.155,82.766,83.277
Figueroa High School,76.403,76.540,76.884,77.151
Ford High School,77.361,77.672,76.918,76.180
Griffin High School,82.044,84.229,83.842,83.356
Hernandez High School,77.438,77.337,77.136,77.187
Holden High School,83.787,83.430,85.000,82.855
Huang High School,77.027,75.909,76.447,77.226
Johnson High School,77.188,76.691,77.492,76.863


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [9]:
grade9_read_scores = grade9["reading_score"].mean()
grade10_read_scores = grade10["reading_score"].mean()
grade11_read_scores = grade11["reading_score"].mean()
grade12_read_scores = grade12["reading_score"].mean()

read_df = pd.DataFrame({
    "9th": grade9_read_scores.apply(lambda x: "{:.3f}".format(x)),
    "10th": grade10_read_scores.apply(lambda x: "{:.3f}".format(x)),
    "11th": grade11_read_scores.apply(lambda x: "{:.3f}".format(x)),
    "12th": grade12_read_scores.apply(lambda x: "{:.3f}".format(x))
})
read_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303,80.907,80.946,80.912
Cabrera High School,83.676,84.253,83.788,84.288
Figueroa High School,81.199,81.409,80.640,81.385
Ford High School,80.633,81.263,80.404,80.662
Griffin High School,83.369,83.707,84.288,84.014
Hernandez High School,80.867,80.660,81.396,80.857
Holden High School,83.677,83.325,83.816,84.699
Huang High School,81.290,81.512,81.417,80.306
Johnson High School,81.261,80.773,80.616,81.228


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [10]:
# The range among Per Student Spending is $77/student.
# The data can fit into 4 bins spanning $20.
spending_bins = [0, 599, 619, 639, 660]
spending_labels = ["<$600", "$600-619", "$620-639", ">$640"]

In [11]:
spending_df = pd.DataFrame({
    "Per Student Budget": School_Sum_df["Per Student Budget"],
    "Total Students": School_Sum_df["Total Students"],
    
    # Formatting the School Summary data stores the data as strings.
    # Cast the data in each column to numeric data.
    "Average Math Score": pd.to_numeric(School_Sum_df["Average Math Score"]),
    "Average Reading Score": pd.to_numeric(School_Sum_df["Average Reading Score"]),
    "% Passing Math": pd.to_numeric(School_Sum_df["% Passing Math"]),
    "% Passing Reading": pd.to_numeric(School_Sum_df["% Passing Reading"]),
    "% Overall Passing Rate": pd.to_numeric(School_Sum_df["Overall Passing Rate"])
})

# Place the data into bins according to "Per Student Budget".
spending_df["Spending Ranges (Per Student)"] = pd.cut(budget_per_student_df, spending_bins, labels=spending_labels)
spending_groupby = spending_df.groupby("Spending Ranges (Per Student)")
spending_groupby.mean()


,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,,
<$600,1592.0,83.45525,83.93375,93.46000,96.6110,95.0355
$600-619,1361.5,83.59950,83.88550,94.23100,95.9005,95.0655
$620-639,3005.4,79.47440,82.12060,77.13980,87.4680,82.3042
>$640,3763.0,77.02350,80.95725,66.70125,80.6750,73.6880


## Scores by School Size

* Perform the same operations as above, based on school size.

In [12]:
# All District schools have more than 2000 schools.
# An extra category is added to further distinguish
# District schools.
size_bins = [0, 1000, 2000, 3000, 5000]
size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-3000)", "Extra Large (3000-5000)"]

In [13]:
spending_df["School Size"] = pd.cut(spending_df["Total Students"], size_bins, size_labels)
size_groupby = spending_df.groupby("School Size")
size_groupby.mean()

,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,,
"(0, 1000]",694.50,83.82150,83.93000,93.5505,96.0995,94.8245
"(1000, 2000]",1704.40,83.37440,83.86460,93.5994,96.7908,95.1954
"(2000, 3000]",2722.00,78.42950,81.76900,73.4625,84.4735,78.9680
"(3000, 5000]",4592.75,77.06325,80.91975,66.4645,81.0595,73.7620


## Scores by School Type

* Perform the same operations as above, based on school type.

In [14]:
type_bins = ["District", "Charter"]
type_labels = ["District", "Charter"]

In [15]:
spending_df["School Type"] = School_Sum_df["School Type"]
type_groupby = spending_df.groupby("School Type")
type_groupby.mean()

,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,,
Charter,1524.250000,83.473625,83.896500,93.620750,96.586625,95.103750
District,3853.714286,76.956714,80.966571,66.548571,80.798857,73.673714


Since the correlation between Passing Rates and school type is 
so strong, the following sorted data frames are used
to determine trends beyond school type:

In [16]:
spending_df.sort_values("Per Student Budget")

,Per Student Budget,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate,Spending Ranges (Per Student),School Size,School Type
School Name,,,,,,,,,,
Wilson High School,$578.00,2283,83.274,83.989,93.868,96.540,95.204,<$600,"(2000, 3000]",Charter
Holden High School,$581.00,427,83.803,83.815,92.506,96.253,94.379,<$600,"(0, 1000]",Charter
Cabrera High School,$582.00,1858,83.062,83.976,94.133,97.040,95.587,<$600,"(1000, 2000]",Charter
Wright High School,$583.00,1800,83.682,83.955,93.333,96.611,94.972,<$600,"(1000, 2000]",Charter
Shelton High School,$600.00,1761,83.359,83.726,93.867,95.855,94.861,$600-619,"(1000, 2000]",Charter
Pena High School,$609.00,962,83.840,84.045,94.595,95.946,95.270,$600-619,"(0, 1000]",Charter
Griffin High School,$625.00,1468,83.351,83.817,93.392,97.139,95.266,$620-639,"(1000, 2000]",Charter
Bailey High School,$628.00,4976,77.048,81.034,66.680,81.933,74.307,$620-639,"(3000, 5000]",District
Rodriguez High School,$637.00,3999,76.843,80.745,66.367,80.220,73.293,$620-639,"(3000, 5000]",District


In [17]:
spending_df.sort_values("Total Students")

,Per Student Budget,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate,Spending Ranges (Per Student),School Size,School Type
School Name,,,,,,,,,,
Holden High School,$581.00,427,83.803,83.815,92.506,96.253,94.379,<$600,"(0, 1000]",Charter
Pena High School,$609.00,962,83.840,84.045,94.595,95.946,95.270,$600-619,"(0, 1000]",Charter
Griffin High School,$625.00,1468,83.351,83.817,93.392,97.139,95.266,$620-639,"(1000, 2000]",Charter
Thomas High School,$638.00,1635,83.418,83.849,93.272,97.309,95.291,$620-639,"(1000, 2000]",Charter
Shelton High School,$600.00,1761,83.359,83.726,93.867,95.855,94.861,$600-619,"(1000, 2000]",Charter
Wright High School,$583.00,1800,83.682,83.955,93.333,96.611,94.972,<$600,"(1000, 2000]",Charter
Cabrera High School,$582.00,1858,83.062,83.976,94.133,97.040,95.587,<$600,"(1000, 2000]",Charter
Wilson High School,$578.00,2283,83.274,83.989,93.868,96.540,95.204,<$600,"(2000, 3000]",Charter
Ford High School,$644.00,2739,77.103,80.746,68.310,79.299,73.804,>$640,"(2000, 3000]",District


In [18]:
spending_df.sort_values("% Overall Passing Rate")

,Per Student Budget,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate,Spending Ranges (Per Student),School Size,School Type
School Name,,,,,,,,,,
Rodriguez High School,$637.00,3999,76.843,80.745,66.367,80.220,73.293,$620-639,"(3000, 5000]",District
Figueroa High School,$639.00,2949,76.712,81.158,65.988,80.739,73.364,$620-639,"(2000, 3000]",District
Huang High School,$655.00,2917,76.629,81.183,65.684,81.316,73.500,>$640,"(2000, 3000]",District
Johnson High School,$650.00,4761,77.072,80.966,66.058,81.222,73.640,>$640,"(3000, 5000]",District
Ford High School,$644.00,2739,77.103,80.746,68.310,79.299,73.804,>$640,"(2000, 3000]",District
Hernandez High School,$652.00,4635,77.290,80.934,66.753,80.863,73.808,>$640,"(3000, 5000]",District
Bailey High School,$628.00,4976,77.048,81.034,66.680,81.933,74.307,$620-639,"(3000, 5000]",District
Holden High School,$581.00,427,83.803,83.815,92.506,96.253,94.379,<$600,"(0, 1000]",Charter
Shelton High School,$600.00,1761,83.359,83.726,93.867,95.855,94.861,$600-619,"(1000, 2000]",Charter
